In [74]:
import sys
sys.path += ['/Users/cswa648/git/phylo-hacking/pymc']
import theano
%matplotlib inline

In [79]:
from xml.etree import ElementTree

data_xml_filename = '../../data/ratites.SRD06.RLC.YULE.xml'
xml_root = ElementTree.parse(data_xml_filename)

In [78]:
import newick

newick_string = xml_root.find('.//newick').text
tree = newick.loads(newick_string)[0]
print(tree.ascii_art())

                            ┌─ANDI
                     ┌──────┤
                     │      └─DIGI
              ┌──────┤
              │      │             ┌─CASS
              │      │      ┌──────┤
       ┌──────┤      └──────┤      └─EMU
       │      │             └─KIWI
───────┤      │
       │      └─RHEA
       └─OST


In [87]:
sequence_dict = { seq_element.find('./taxon').attrib['idref']: list(seq_element.itertext())[-1].strip()
 for seq_element in xml_root.findall('.//sequence') }
list(sequence_dict.items())[0]

('ANDI',
 'ATGAACCTAACACTCTTCGACCAATTCTCCAGCCCTCAATTACTAGGCATCCCCCTCATCCTACTCTCCACGCTATTCCCCGCCCTTCTTCTACCATCCCCCAACAACCGCTGAATCACTAACCGACTTTCAGCCCTCCAACTATGACTCTTCCAACTTATCACCAAACAACTCATAATGCCACTAAACAAAGCAGGACACAACTGAGCTCTCATTCTCTCTTCGCTAATAACCTTCCTCCTACTAATTAACCTGCTAGGACTCCTTCCATACACATTCACCCCCACCACCCAACTATCCATAAACATGGCACTAGCCTTCCCCCTCTGACTAGCAACCCTTCTTACAGGACTACGAAACCAACCATCTATCTCCCTAGCCCATATTCTACCCGAAGGTACTCCCACACCCCTAATCCCAGCTTTAATCCTTATCGAAACAACAAGTCTACTTATCCGACCACTAGCCCTAGGAGTCCGTCTCACAGCAAACCTAACAGCCGGCCACCTCTTAATCCAACTTATCTCTACCGCAACCATAGCCCTCCTCTCCATTATACCCACCATCTCAATCCTAACAATACTTATCCTGCTCCTACTAACAATCCTAGAACTAGCAGTAGCCATAATCCAAGCCTACGTCTTCGTACTTCTCCTAAGCTTATATTTACAAGAAAATATCATGCCTCAACTAAACCCACACCCATGATTCTCCATCTTACTTATAGTATGATTCATCCTCCTACTTCTCATCCAGCCTAAATTTCTTTCCTTTATCTACACAAACCTCCCATCCAACAAGACCAAAATAGCCCCTATCCCCCCTACATGAACCTGACCATGAACCGTGACATTCATTACCCGATGACTCTTTTCAACCAACCATAAGGACATTGGCACATTATACCTAATCTTCGGCGCATGAGCAGGCATAGTAGGCACAGCCCTCAGCCTACTCATCCGGGCTGAGCTCGGCCAACCAGGGACCCT

In [94]:
from importlib import reload
import pylo.tree
reload(pylo.tree)
import pylo.topology
reload(pylo.topology)
from pylo.topology import TreeTopology

topology = pylo.topology.TreeTopology(tree)
topology.leaf_children

array([['ANDI', 'DIGI'],
       ['CASS', 'EMU'],
       [None, 'KIWI'],
       [None, None],
       [None, 'RHEA'],
       [None, 'OST']], dtype=object)

In [111]:
import numpy as np
from pylo.transform import group_sequences, encode_sequences

sequence_dict_encoded = encode_sequences(sequence_dict)
pattern_dict, pattern_counts = group_sequences(sequence_dict_encoded)
child_patterns = np.array(topology.build_sequence_table(pattern_dict))
child_patterns.shape, child_patterns.dtype

((6, 2, 922), dtype('int64'))

In [112]:
pattern_counts = np.array(pattern_counts)
pattern_counts.shape, pattern_counts.dtype

((922,), dtype('int64'))

In [116]:
import theano.tensor as tt

child_patterns_ = tt.as_tensor_variable(child_patterns)
pattern_counts_ = tt.as_tensor_variable(pattern_counts)

In [119]:
topology.leaf_mask

array([[ True,  True],
       [ True,  True],
       [False,  True],
       [False, False],
       [False,  True],
       [False,  True]])

In [127]:
tt.as_tensor_variable(topology.child_indices)

TensorConstant{[[-1 -1]
 ..
 [ 4 -1]]}

In [184]:
import pymc3 as pm

from pylo.tree import BirthDeathSamplingTree, TreeHeightProportionTransform
from pylo.hky import HKYSubstitutionModel

import pylo.pruning
reload(pylo.pruning)

from pylo.pruning import LeafSequences

with pm.Model() as model:
    r = pm.Exponential('r', lam=0.1)
    a = pm.Beta('a', alpha=1.0, beta=1.0)
    rho = pm.Beta('rho', alpha=1.0, beta=1.0)
    
    tree_heights = BirthDeathSamplingTree('tree', topology, r, a, rho)
    branch_lengths = pm.Deterministic('branch_lengths', topology.get_child_branch_lengths(tree_heights))
    rates = pm.Lognormal('rates', shape=(topology.get_internal_node_count(), 2))
    
    distances = pm.Deterministic('distances', rates*branch_lengths)
    
    kappa = pm.Exponential('kappa', lam=0.1)
    pi = pm.Dirichlet('pi', a=np.ones(4))
    
    substitution_model = HKYSubstitutionModel(kappa, pi)
    sequences = LeafSequences('sequences', topology, substitution_model, distances, child_patterns_, pattern_counts_)
    
model

In [156]:
for RV in model.basic_RVs:
    print(RV.name, RV.logp(model.test_point))

r_log__ -1.0596601002984285
a_logodds__ -1.3862943611198906
rho_logodds__ -1.3862943611198906
tree -21.995558354525762
rates_log__ -11.027262398456068
kappa_log__ -1.0596601002984285
pi_stickbreaking__ -3.753417975251508


In [130]:
with model:
    trace = pm.sample(chains=1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO (theano.gof.compilelock): Refreshing lock /Users/cswa648/.theano/compiledir_Darwin-17.6.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir/lock
Sequential sampling (1 chains in 1 job)
NUTS: [pi, kappa, rates, tree, rho, a, r]
  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: Bad initial energy: nan. The model might be misspecified.